In [33]:
%%bash

pip install gymnasium==0.29.1
pip install gymnasium[atari]
pip install gymnasium[accept-rom-license]

  Using cached ale_py-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
Using cached ale_py-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
  Attempting uninstall: ale-py
    Found existing installation: ale-py 0.10.1
    Uninstalling ale-py-0.10.1:
      Successfully uninstalled ale-py-0.10.1


In [34]:
import gymnasium as gym; print(f'Gymnasium v{gym.__version__}')

Gymnasium v0.29.1


In [35]:
%%bash

pip install "stable-baselines3[extra]>=2.0.0a4"
pip install sb3-contrib

  Using cached ale_py-0.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.6 kB)
Using cached ale_py-0.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Attempting uninstall: ale-py
    Found existing installation: ale-py 0.8.1
    Uninstalling ale-py-0.8.1:
      Successfully uninstalled ale-py-0.8.1


In [39]:
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
import numpy as np
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import VecEnv, SubprocVecEnv
from stable_baselines3.common.vec_env import VecEnv, SubprocVecEnv
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor, NatureCNN, create_mlp
import torch as th
from torch import nn
from sb3_contrib.common.recurrent.policies import RecurrentActorCriticPolicy
import crafter

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
class NotebookVecEnvWrapper(VecEnv):
    def __init__(self, venv, num_envs, notebook_size=8):
        self.venv = venv
        self.num_envs = num_envs
        # if type(venv.observation_space) != spaces.Discrete:
        #     raise ValueError("Only Discrete observation spaces are supported for now")
        # if type(venv.action_space) != spaces.Discrete:
        #     raise ValueError("Only Discrete action spaces are supported for now")

        self.notebook_size = notebook_size
        self.notebook = np.ones(self.notebook_size, dtype=np.int32)

        self.observation_space = spaces.Dict({
                                  "env": venv.observation_space,  # Original Observation Space
                                  "notebook": spaces.MultiDiscrete([2] * self.notebook_size)
                                 })
        self.action_space = spaces.MultiDiscrete([venv.action_space.n, self.notebook_size, 2])
        print(f"Observation Space: {self.observation_space}")
        print(f"Action Space: {self.action_space}")

    def reset(self):
        observations = self.venv.reset()
        #observations = observations[np.newaxis, ...]
        #observations = observations.transpose(0, 3, 1, 2)
        single_env_shape = (84, 84, 4)
        observation_space = spaces.Box(low=0, high=255, shape=single_env_shape, dtype=np.uint8)
        observations = np.random.randint(0, 256, size=(self.num_envs, *single_env_shape), dtype=np.uint8)
        batch_observation_space = spaces.Box(low=0, high=255, shape=(self.num_envs, *single_env_shape), dtype=np.uint8)
        self.notebook = np.ones(self.notebook_size, dtype=np.int32)
        # print(f"Reset Result is {observations.shape}")
        # print(f"VENV_OBS is {type(venv_obs)}")
        reset_observation = {
                            "env": observations,  # Original Observation Space
                            "notebook": self.notebook
                            }
        return reset_observation

    def step(self, action):
        venv_action, notebook_index, notebook_value = action
        venv_obs, reward, done, info = self.venv.step([venv_action])
        self.notebook[notebook_index] = notebook_value
        combined_observation = {
        "env": venv_obs,  # Observations from the original environment
        "notebook": self.notebook,  # Current notebook state
        }
        return combined_observation, reward, done, info

    def env_is_wrapped(self, wrapper_class, indices=None):
        return self.venv.env_is_wrapped(wrapper_class, indices)

    def get_attr(self, attr_name, indices=None):
        return self.venv.get_attr(attr_name, indices)

    def set_attr(self, attr_name, value, indices=None):
        return self.venv.set_attr(attr_name, value, indices)

    def env_method(self, method_name, *method_args, indices=None, **method_kwargs):
        return self.venv.env_method(method_name, *method_args, indices=indices, **method_kwargs)

    def step_async(self, actions):
        self.venv.step_async(actions)

    def step_wait(self):
        return self.venv.step_wait()

    def render(self):
        return self.venv.render()

    def close(self):
        return self.venv.close()

    def seed(self, seed):
        return self.venv.seed(seed)

    def __getattr__(self, attr):
        return getattr(self.venv, attr)

    def __str__(self):
        return str(self.venv)

    def __repr__(self):
        return repr(self.venv)

In [7]:
# There already exists an environment generator that will make and wrap atari environments correctly.
num_envs = 2
venv = make_atari_env("PongNoFrameskip-v4", n_envs=num_envs, seed=0)
# Stack 4 frames
venv = VecFrameStack(venv, n_stack=4)
notebook_venv = NotebookVecEnvWrapper(venv, num_envs)
# notebook_venv.reset()

# print(type(notebook_venv))
# print(notebook_venv.observation_space)
# print(notebook_venv.action_space)

# # Test reset
# reset_obs = notebook_venv.reset()
# print(f"Reset Observation Keys: {reset_obs.keys()}")
# print(f"Env Observation Shape: {reset_obs['env'].shape}")
# print(f"Notebook Observation Shape: {reset_obs['notebook'].shape}")

# # Test step
# # dummy_actions = np.array([notebook_venv.action_space.sample() for _ in range(num_envs)])
# dummy_action = [0, 0, 1]
# step_obs, reward, done, info = notebook_venv.step(dummy_action)
# print(f"Step Observation Keys: {step_obs.keys()}")
# print(f"Env Observation Shape: {step_obs['env'].shape}")
# print(f"Notebook Observation Shape: {step_obs['notebook'].shape}")


Observation Space: Dict('env': Box(0, 255, (84, 84, 4), uint8), 'notebook': MultiDiscrete([2 2 2 2 2 2 2 2]))
Action Space: MultiDiscrete([6 8 2])


In [31]:
class CombinedExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: spaces.Dict):
        super(CombinedExtractor, self).__init__(observation_space, features_dim=128) # Adjust features_dim if needed

        # Assuming 'env' key contains the image observations
        print(f"Observation Space Env: {observation_space.spaces['env'].shape}")
        print(f"Observation Space Notebook: {observation_space.spaces['notebook'].shape}")
        self.cnn = NatureCNN(observation_space.spaces['env'], features_dim=64) # Adjust features_dim if needed
        # Assuming 'notebook' key contains the notebook observations
        notebook_size = observation_space.spaces['notebook'].nvec[0]  # Get size of the notebook
        self.mlp = create_mlp(notebook_size, 64, [64])  # Adjust hidden layer size if needed
        self.mlp = nn.Sequential(*self.mlp)

    def forward(self, observations) -> th.Tensor:
        # Debug raw input shapes
        print(f"Raw env observation shape: {observations['env'].shape}")
        print(f"Raw notebook observation shape: {observations['notebook'].shape}")

        # Extract features from each part of the observation
        encoded_env = self.cnn(observations['env'])
        encoded_notebook = self.mlp(th.Tensor(observations['notebook']))

        # Debug feature shapes
        print(f"encoded_env shape: {encoded_env.shape}")
        print(f"encoded_notebook shape: {encoded_notebook.shape}")

        # Ensure batch sizes match
        if encoded_env.shape[0] != encoded_notebook.shape[0]:
            # Truncate or reshape notebook observations if needed
            encoded_notebook = encoded_notebook[:encoded_env.shape[0]]
            print(f"Adjusted encoded_notebook shape: {encoded_notebook.shape}")

        # Concatenate the features
        return th.cat([encoded_env, encoded_notebook], dim=1)


In [30]:
import torch as th
from stable_baselines3.common.preprocessing import is_image_space, preprocess_obs
from gymnasium import spaces

def preprocess_obs(obs, observation_space, normalize_images=True):
    """
    Preprocess observation to be to a neural network.
    For images, it normalizes the values by dividing by 255.
    For discrete observations, it create a one hot vector.
    :param obs: (np.ndarray)
    :param observation_space: (spaces.Space)
    :param normalize_images: (bool) Whether to normalize images or not,
        dividing by 255.0 if `True` (default)
    :return: (np.ndarray, th.Tensor)
    """
    if isinstance(observation_space, spaces.Dict):
      preprocessed_obs = {}
      for key, _obs in obs.items():
          # Check if the key is 'notebook' to apply special handling
          if key == 'notebook':
              # Convert to one-hot encoding for MultiDiscrete 'notebook' observation
              _obs = _obs.type(th.int64)  # Ensure _obs is a PyTorch tensor with int64 dtype
              num_classes = observation_space[key].nvec[0]  # Assuming all nvec elements are equal
              preprocessed_obs[key] = th.nn.functional.one_hot(_obs, num_classes=num_classes).float()

          else:
              # Preprocess other observations (e.g., images) normally
              preprocessed_obs[key] = preprocess_obs(_obs, observation_space[key], normalize_images=normalize_images)

      return preprocessed_obs
    elif is_image_space(observation_space):
        # Normalize image
        return th.as_tensor(obs / 255.0, dtype=th.float32)
    elif isinstance(observation_space, spaces.Box):
        # For Box spaces, convert to tensor and potentially normalize
        return th.as_tensor(obs, dtype=th.float32)  # No normalization for now
    elif isinstance(observation_space, spaces.Discrete):
        # One hot encoding for Discrete space
        return th.nn.functional.one_hot(th.as_tensor(obs).long(), num_classes=observation_space.n).float()
    elif isinstance(observation_space, spaces.MultiDiscrete):
        # Special handling for MultiDiscrete spaces
        # Convert to tensor and perform one-hot encoding for each dimension
        return th.cat(
            [
                th.nn.functional.one_hot(th.as_tensor(obs_.astype(np.int64)), num_classes=int(observation_space.nvec[idx])).float()
                for idx, obs_ in enumerate(obs)
            ],
            dim=-1,
        )
    elif isinstance(observation_space, spaces.MultiBinary):
        return th.as_tensor(obs, dtype=th.float32)
    else:
        raise NotImplementedError(f"Preprocessing not implemented for {observation_space}")


class CustomRecurrentActorCriticPolicy(RecurrentActorCriticPolicy):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        env_obs_shape = self.observation_space.spaces["env"].shape # Get the shape tuple
        print(f"Env Observation Shape: {env_obs_shape}")
        env_obs_size = env_obs_shape[0] # Calculate size from the shape
        notebook_obs_size = self.observation_space.spaces["notebook"].nvec[0]
        total_input_size = 128
        self.lstm_actor.input_size = total_input_size
        if self.lstm_critic is not None:
            self.lstm_critic.input_size = total_input_size

    def extract_features(self, obs: th.Tensor) -> th.Tensor:
        """
        Preprocess the observation if needed and extract features.

        :param obs:
        :return:
        """
        # Call your custom preprocess_obs function here
        preprocessed_obs = preprocess_obs(obs, self.observation_space, normalize_images=self.normalize_images)
        preprocessed_obs['notebook'] = th.as_tensor(preprocessed_obs['notebook'])
        print(f"preprocessed_obs[env] = {preprocessed_obs['env'].shape}")
        #batch_size = preprocessed_obs['env'].shape[0]
        # Flatten the rest of the dimensions (except batch)
        #preprocessed_obs['env'] = preprocessed_obs['env'].reshape(batch_size, -1)
        return self.features_extractor(preprocessed_obs)

    @staticmethod
    def _process_sequence(features, lstm_states, episode_starts, lstm):
        """
        Process input sequence with the LSTM.

        :param features: Features tensor of shape (batch_size, input_size).
        :param lstm_states: Tuple of initial hidden and cell states (h_0, c_0).
        :param episode_starts: A tensor indicating the start of new episodes.
        :param lstm: The LSTM module to process the sequence.
        :return: Processed sequence and updated LSTM states.
        """
        batch_size, input_size = features.shape
        n_seq = 2  # Number of sequences (adjust as needed)
        max_length = batch_size // n_seq

        # Reshape and transpose to match LSTM input shape (seq_len, batch, input_size)
        features_sequence = features.view((n_seq, max_length, input_size)).swapaxes(0, 1)
        print(f"Features Sequence Shape: {features_sequence.shape}")

        # Process the sequence with the LSTM
        lstm_output, lstm_states = lstm(features_sequence, lstm_states)

        return lstm_output, lstm_states

In [32]:
model = RecurrentPPO(
    CustomRecurrentActorCriticPolicy,
    notebook_venv,
    verbose=1,
    policy_kwargs={
        "features_extractor_class": CombinedExtractor,
        "features_extractor_kwargs": {}, # Optional: You can pass extra kwargs to the extractor
        "net_arch": [dict(pi=[64, 64], vf=[64, 64], lstm=[128, 128])], # Adjust net_arch if needed
        "lstm_hidden_size": 128,
    },
)
model.learn(2000000)

vec_env = model.get_env()
mean_reward, std_reward = evaluate_policy(model, vec_env, n_eval_episodes=20, warn=False)
print(mean_reward)

Using cuda device
Wrapping the env in a VecTransposeImage.
Observation Space Env: (4, 84, 84)
Observation Space Notebook: (8,)
Env Observation Shape: (4, 84, 84)
preprocessed_obs[env] = torch.Size([2, 4, 84, 84])
Raw env observation shape: torch.Size([2, 4, 84, 84])
Raw notebook observation shape: torch.Size([8, 2])
encoded_env shape: torch.Size([2, 64])
encoded_notebook shape: torch.Size([8, 64])
Adjusted encoded_notebook shape: torch.Size([2, 64])
Features Sequence Shape: torch.Size([1, 2, 128])
Features Sequence Shape: torch.Size([1, 2, 128])


RuntimeError: split_with_sizes expects split_sizes to sum exactly to 2 (input tensor's size at dimension 1), but got split_sizes=[6, 8, 2]